# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries in XML document
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
# Had to add following string because 'variable is not named' error occurred...
capitals_string = ''
for element in document_tree.iterfind('country'):
    #print('* ' + element.find('name').text + ':' + capitals_string = '')
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë, Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë, Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes, Skopje, Kumanovo
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë, Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes, Skopje, Kumanovo, Beograd, Novi Sad, Niš
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë, Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloni

****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
import pandas as pd
document = ET.parse( './data/mondial_database.xml' )
document_root = document.getroot()

### Part 1: Find 10 countries with the lowest infant mortality rates.

In [6]:
# dataframe for the population measurement with year and country 
df_infant_mortality = pd.DataFrame(columns=["country", "infant_mortality"])

# iterate over xml - tree and extract
# country and infant mortality
for country in document_root.getiterator("country"):
    if country.findall("infant_mortality") is None:
        continue;
        
    for elem in country.findall("infant_mortality"):
        infant_mortality = elem.text
            
        df_infant_mortality = df_infant_mortality.append(
        {
            "country": country.find("name").text,
            "infant_mortality": infant_mortality
        }, ignore_index=True)    

# Checking the results
df_infant_mortality.head()

,country,infant_mortality
0,Albania,13.19
1,Greece,4.78
2,Macedonia,7.9
3,Serbia,6.16
4,Andorra,3.69


In [7]:
df_infant_mortality["infant_mortality"] = pd.to_numeric(df_infant_mortality["infant_mortality"])
b = df_infant_mortality.sort_values("infant_mortality")

# 10 countries with the lowest infant mortalities, presented.
b.head(10)

,country,infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


### Part 2: Find 10 cities with the largest population

In [8]:
# dataframe for the population measurement with year and country 
df_city_pop = pd.DataFrame(columns=["city", "year", "population"])

# iterate over xml - tree and extract
# city, year and population
for country in document_root.getiterator("country"):
    for city in country.iter("city"):
        city_name = city.find("name").text
        
        if city.findall("population") is None:
            continue;
        
        for pop in city.findall("population"):
            year = pop.attrib["year"]
            population = pop.text
            
            df_city_pop = df_city_pop.append(
            {
                "city": city_name,
                "year": year,
                "population": population
            }, ignore_index=True)
            

In [9]:
# Sorting and filtering, or vice-versa
c = df_city_pop 
c.population = pd.to_numeric(c.population)
c = c.sort_values("population", ascending=False)

# Getting rid of any duplicates - drop everything but the first duplicate
c = c.drop_duplicates(["city"], keep='first')

# Showing off the data
c.head(10)

,city,year,population
3750,Shanghai,2010,22315474
2607,Istanbul,2012,13710512
4398,Delhi,2001,12877470
4303,Mumbai,2011,12442373
1546,Moskva,2013,11979529
3746,Beijing,2010,11716620
8208,São Paulo,2010,11152344
3754,Tianjin,2010,11090314
3364,Guangzhou,2010,11071424
3371,Shenzhen,2010,10358381


### Part 3: Find 10 ethinic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [10]:
# Code courtesy of Andrew Maguire - because I had no idea how to use pandas with XML!
# Thanks, Andrew.
# dataframe for fraction of each ethnic group in country
df_ethnic_groups_fraction = pd.DataFrame(columns=["country", "ethnicgroup", "fraction"])
document_root = document.getroot()

for country in document_root.getiterator("country"):
    for element in list(country):
        if element.tag == "ethnicgroup":
            df_ethnic_groups_fraction = df_ethnic_groups_fraction.append({
                "country": country.find("name").text,
                "ethnicgroup": element.text,
                "fraction": element.attrib["percentage"]
            }, ignore_index=True)
            
# convert fraction to numeric
df_ethnic_groups_fraction["fraction"] = pd.to_numeric(df_ethnic_groups_fraction["fraction"])

# get fraction between 0 and 1
df_ethnic_groups_fraction["fraction"] = df_ethnic_groups_fraction["fraction"] / 100

d = df_ethnic_groups_fraction.sort_values("fraction", ascending=False)
d.head(10)

,country,ethnicgroup,fraction
627,Seychelles,Seychellois,1.0
452,Wallis and Futuna,Polynesian,1.0
152,Guernsey,Norman-French,1.0
153,Iceland,Celt,1.0
157,Jersey,Norman-French,1.0
368,Dominica,Carib Indians,1.0
384,Grenada,African,1.0
402,Saint Kitts and Nevis,African,1.0
405,Saint Pierre and Miquelon,Basques Bretons,1.0
422,Kiribati,Micronesian,1.0


### Part 4: Name and country of a) longest river, b) largest lake and c) airport at highest elevation
*longest river*

In [11]:
# dataframe for the population measurement with year and country 
df_rivers = pd.DataFrame(columns=["country_code", "river", "river_length"])

# country, river and river_length. Country is an attribute.
# Name and length are text values.
for river in document_root.getiterator("river"):
    if river.find("length") is None:
        continue;
    country = river.attrib["country"]
    river_length = river.find('length').text
            
    df_rivers = df_rivers.append(
    {
        "country_code": country,
        "river": river.find("name").text,
        "river_length": river_length
    }, ignore_index=True)

# Finding max river
df_rivers.river_length = pd.to_numeric(df_rivers.river_length)
e = df_rivers.sort_values("river_length", ascending=False)
e.head(1)

,country_code,river,river_length
174,CO BR PE,Amazonas,6448.0


*largest lake*

In [12]:
# dataframe for the population measurement with year and country 
df_lakes = pd.DataFrame(columns=["country_code", "lake", "lake_area"])

# Country is an attribute.
# Name and area are text values.
for lake in document_root.getiterator("lake"):
    if lake.find("area") is None:
        continue;
    country = lake.attrib["country"]
    lake_area = lake.find('area').text
            
    df_lakes = df_lakes.append(
    {
        "country_code": country,
        "lake": lake.find("name").text,
        "lake_area": lake_area
    }, ignore_index=True)

# Finding max river
df_lakes.lake_area = pd.to_numeric(df_lakes.lake_area)
f = df_lakes.sort_values("lake_area", ascending=False)
f.head(1)

,country_code,lake,lake_area
54,R AZ KAZ IR TM,Caspian Sea,386400.0


*airport at highest elevation* 

In [13]:
# dataframe for the population measurement with year and country 
df_airports = pd.DataFrame(columns=["country_code", "airport", "elevation"])

# Country is an attribute.
# Name and elelvation are text values.
for ap in document_root.getiterator("airport"):
    if ap.find("elevation") is None:
        continue;
    country = ap.attrib["country"]
    elevation = ap.find('elevation').text
            
    df_airports = df_airports.append(
    {
        "country_code": country,
        "airport": ap.find("name").text,
        "elevation": elevation
    }, ignore_index=True)

# Finding max river
df_airports.elevation = pd.to_numeric(df_airports.elevation)
g = df_airports.sort_values("elevation", ascending=False)
g.head(1)
# Fun fact: I was born there!

,country_code,airport,elevation
80,BOL,El Alto Intl,4063.0
